<div style="
  background: linear-gradient(90deg, #222e42ff, #d8aee4ff);
  padding: 16px;
  border-radius: 8px;
  color: #ffffff;
  font-family: Arial, sans-serif;
">

  <h2 style="margin: 0 0 8px 0;">
    Pract – 4: Implementation of Federated Learning using PyTorch
  </h2>

  <p style="margin: 0; font-size: 14px; line-height: 1.5;">
    • Generate a synthetic dataset for Linear Regression using PyTorch<br/>
    ( Create 3 clients and train local PyTorch models<br/>
    • Client 1 data: 1, 2, 3, 4, 5<br/>
    • Client 2 data: 2, 4, 6, 8, 10<br/>
    • Client 3 data: 3, 6, 9, 12, 15 )<br/>
    • Aggregate model weights using Federated Averaging<br/>
    • Create a global PyTorch model<br/>
    • Send updated global model back to clients
  </p>

</div>


## Step 1: Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

## Step 2: Generate Synthetic Dataset

In [2]:
# Client datasets

X1 = torch.tensor([[1.0],[2.0],[3.0],[4.0],[5.0]])
y1 = 2 * X1

X2 = torch.tensor([[2.0],[4.0],[6.0],[8.0],[10.0]])
y2 = 2 * X2

X3 = torch.tensor([[3.0],[6.0],[9.0],[12.0],[15.0]])
y3 = 2 * X3

print("Datasets Created Successfully")

Datasets Created Successfully


## Step 3: Define Linear Regression Model

In [3]:
class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1,1)
    
    def forward(self,x):
        return self.linear(x)

## Step 4: Train Local Models

In [4]:
def train_local_model(X, y, epochs=100):
    model = LinearModel()
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    for _ in range(epochs):
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

    return model

In [5]:
model1 = train_local_model(X1, y1)
model2 = train_local_model(X2, y2)
model3 = train_local_model(X3, y3)

## Step 5: Federated Averaging (Aggregate Weights)

In [6]:
def federated_average(models):
    global_model = LinearModel()
    global_dict = global_model.state_dict()

    for key in global_dict:
        global_dict[key] = torch.mean(
            torch.stack([model.state_dict()[key] for model in models]), dim=0
        )

    global_model.load_state_dict(global_dict)
    return global_model

In [7]:
global_model = federated_average([model1, model2, model3])

print("Global Model Created")

Global Model Created


## Step 6: Test Global Model

In [8]:
test_data = torch.tensor([[7.0],[14.0]])
predictions = global_model(test_data)

print("Test Data:\n", test_data)
print("Predictions:\n", predictions.detach().numpy())

Test Data:
 tensor([[ 7.],
        [14.]])
Predictions:
 [[11.895618]
 [23.878086]]
